In [65]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
# Import Keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import sys

In [66]:
train = pd.read_json('train.json')
target_train=train['is_iceberg']
test = pd.read_json('test.json')
batch_size = 32
train.shape

(1604, 5)

In [67]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [68]:
Xtrain = get_scaled_imgs(train)
Xtrain_angle = train['inc_angle']
Xtest = get_scaled_imgs(test)
Xtest_angle = test['inc_angle']
Xtrain.shape
Xtest.shape

(8424, 75, 75, 3)

In [69]:
Ytrain = np.array(train['is_iceberg'])
print(Ytrain.shape)
Xtrain_angle = np.array(Xtrain_angle)
print(Xtrain_angle.shape)
Xtest_angle = np.array(Xtest_angle)


(1604,)
(1604,)


In [70]:
train.inc_angle = train.inc_angle.replace('na',0)
test.inc_angle = test.inc_angle.replace('na',0)
idx_tr = np.where(train.inc_angle>0)
idx_te = np.where(test.inc_angle>0)
idx_tr[0]
idx_te[0]

array([   0,    1,    2, ..., 8421, 8422, 8423])

In [71]:
Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xtest = Xtest[idx_te[0],...]
Xtrain_angle = Xtrain_angle[idx_tr[0],...]
Xtest_angle = Xtest_angle[idx_te[0],...]
print(Ytrain.shape)
print(Xtrain.shape)
print(Xtest.shape)

(1471,)
(1471, 75, 75, 3)
(8424, 75, 75, 3)


In [72]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [73]:
Xtr_more = get_more_images(Xtrain) 
Xtr_more.shape

(4413, 75, 75, 3)

In [74]:
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))
Ytr_more.shape

(4413,)

In [75]:
Xtr_more_angle = np.concatenate((Xtrain_angle,Xtrain_angle,Xtrain_angle))
Xtr_more_angle.shape

(4413,)

In [76]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

#  This kernel is just used for testing

In [61]:
#shape: 32*75*75*3
genX1 = gen.flow(Xtr_more,Ytr_more,  batch_size=batch_size,seed=55)
#shape: 32*75*75*3
genX2 = gen.flow(Xtr_more,Xtr_more_angle, batch_size=batch_size,seed=55)
print(genX2[0])
while True:
            X1i = genX1.next()
            X2i = genX2.next()

(array([[[[  2.87963580e-02,   1.89089365e-02,   3.27246264e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967690e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967690e-02],
         ..., 
         [  1.36803001e-01,   4.71025817e-02,   1.23630926e-01],
         [  1.67214930e-01,   1.16077252e-02,   1.16868265e-01],
         [  9.46181267e-02,  -9.58846360e-02,  -1.02016255e-02]],

        [[  2.87963580e-02,   1.89089365e-02,   3.27246264e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967690e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967690e-02],
         ..., 
         [  1.36803001e-01,   4.71025817e-02,   1.23630926e-01],
         [  1.67214930e-01,   1.16077252e-02,   1.16868265e-01],
         [  9.46181267e-02,  -9.58846360e-02,  -1.02016255e-02]],

        [[  2.87963580e-02,   1.89089365e-02,   3.27246264e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967690e-02],
         [ -1.28531516e-01,  -2.02839058e-02,  -9.82967

KeyboardInterrupt: 

In [77]:
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

In [78]:
# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

Layer: block5_pool:(MaxPooling2D):(None, 7, 7, 512) parameter: 0 

In [38]:
VGG16().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [39]:
MobileNet().summary()

17227776/17225924 [==============================] - 4s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_r

### This kernel just used for test

In [60]:
#Xtr_more.shape[1:]
base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=Xtr_more.shape[1:], classes=1)
x = base_model.get_layer('block5_pool').output
x = GlobalMaxPooling2D()(x)
print(x)
base_model2 = MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,
                        include_top=False, input_shape=Xtr_more.shape[1:])
x2 = base_model2.output
print(x2)
x2 = GlobalAveragePooling2D()(x2)
input_2 = Input(shape=[1], name="angle")
print(input_2)
angle_layer = Dense(1, )(input_2)
print(angle_layer)
merge_one = concatenate([x, x2, angle_layer])
print(merge_one)

Tensor("global_max_pooling2d_13/Max:0", shape=(?, 512), dtype=float32)
Tensor("conv_pw_13_relu_11/clip_by_value:0", shape=(?, 3, 3, 921), dtype=float32)
Tensor("angle_4:0", shape=(?, 1), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 1), dtype=float32)
Tensor("concatenate_2/concat:0", shape=(?, 1434), dtype=float32)


In [79]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=Xtr_more.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,
                            include_top=False, input_shape=Xtr_more.shape[1:])
    #base_model3 = ResNet50()
    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])

    merge_one = Dropout(0.2)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    adam = Adam(lr=1e-3) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

In [81]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, Ytr_more))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = Xtr_more[train_idx]
        y_train_cv = Ytr_more[train_idx]
        X_holdout = Xtr_more[test_idx]
        Y_holdout= Ytr_more[test_idx]
        
        #Angle
        X_angle_cv=Xtr_more_angle[train_idx]
        X_angle_hold=Xtr_more_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([Xtest, Xtest_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([Xtr_more, Xtr_more_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(Ytr_more, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(Ytr_more, y_valid_pred_log))
    return y_test_pred_log

In [82]:
preds=myAngleCV(Xtr_more, Xtr_more_angle, Xtest)
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)


===================FOLD= 0


/u/13/lix12/unix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 93s 4s/step - loss: 0.7613 - acc: 0.5547 - val_loss: 0.6807 - val_acc: 0.5130
Epoch 2/100
24/24 [==============================] - 87s 4s/step - loss: 0.7311 - acc: 0.5339 - val_loss: 0.6760 - val_acc: 0.4881
Epoch 3/100
24/24 [==============================] - 86s 4s/step - loss: 0.6663 - acc: 0.6172 - val_loss: 0.7053 - val_acc: 0.4881
Epoch 4/100
24/24 [==============================] - 84s 4s/step - loss: 0.7043 - acc: 0.5768 - val_loss: 0.7098 - val_acc: 0.4881
Epoch 5/100
24/24 [==============================] - 84s 4s/step - loss: 0.6696 - acc: 0.6111 - val_loss: 0.6840 - val_acc: 0.4881
Epoch 6/100
24/24 [==============================] - 90s 4s/step - loss: 0.6478 - acc: 0.6523 - val_loss: 0.6160 - val_acc: 0.4881
Epoch 7/100
24/24 [==============================] - 86s 4s/step - loss: 0.6451 - acc: 0.6497 - val_loss: 0.6973 - val_acc: 0.4881
Epoch 8/100
24/24 [==============================] - 85s 4s/step - loss: 0.6767 - a

ValueError: [1606 1608 1613 1619 1620 1636 1646 1647 1648 1654 1659 1661 1682 1688 1692
 1696 1697 1706 1719 1725 1741 1757 1761 1764 1782 1787 1788 1802 1810 1811
 1813 1815 1821 1822 1828 1835 1836 1851 1854 1855 1859 1861 1865 1871 1873
 1882 1889 1891 1896 1901 1902 1905 1908 1912 1913 1917 1927 1928 1934 1942
 1947 1958 1963 1965 1969 1975 1976 1977 1983 1988 1991 1998 1999 2003 2013
 2021 2022 2026 2030 2036 2037 2039 2049 2052 2064 2068 2070 2071 2072 2076
 2077 2086 2089 2091 2096 2098 2100 2103 2108 2111 2118 2119 2130 2132 2140
 2144 2156 2159 2163 2165 2173 2176 2187 2190 2209 2210 2215 2228 2231 2234
 2257 2259 2267 2269 2275 2277 2283 2290 2291 2293 2297 2301 2304 2306 2307
 2311 2313 2317 2330 2345 2352 2363 2364 2365 2375 2376 2378 2380 2381 2397
 2400 2403 2406 2410 2415 2418 2419 2421 2425 2431 2435 2437 2451 2454 2455
 2458 2460 2461 2466 2471 2473 2478 2483 2485 2490 2496 2500 2505 2507 2514
 2526 2537 2548 2560 2565 2572 2576 2579 2582 2593 2602 2607 2609 2612 2618
